In [14]:
"""
MODIFICADO DE cpv2010arg.py en https://github.com/abenassi/pyredatam/blob/1480c481feb0698d54b59c3c17e52661a8c793df/pyredatam/cpv2010arg.py
"""
import pandas as pd
import numpy as np
import os

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import bs4
from bs4 import BeautifulSoup
import requests

import time
import import_ipynb
from Redapy_query import query_final ## el código esta en el notebook llamado Redapy_query
import redapy

area = ["Provinci"]
var1 = ["Poblacio.PAREA"]
var2 = ["Poblacio.P02SEXO"]
selection = [""]
filter_a = ""

query = query_final(tipo="crosstab",variables1=var1,variables2=var2,area_break=area)
print(query)
ser = Service(r'C:\\Users\\Dieguinchi\\Desktop\\chromedriver.exe')
BASE_URL = "http://censos.inei.gob.pe/cgibin/RpWebEngine.exe/CmdSet?&BASE=CPV2007"

def make_query(query, url=BASE_URL): # hace consulta "query" a redatam a través de procesador estadístico online
    options = webdriver.ChromeOptions()#carga configuración del webdriver
    driver = webdriver.Chrome(options=options, service=ser)# el driver debe estar en archivos del pc. puede descargarse de https://chromedriver.chromium.org/downloads
    driver.get(url) # abre pagina web de redatam
    query_input = driver.find_element(By.TAG_NAME,"textarea")# ubica linea de comandos
#     query_input.send_keys(query.decode("utf-8", "ignore"))
    query_input.send_keys(query) # escribe consulta en línea de comandos
    submit = driver.find_element(By.NAME,"SUBMIT") #Busca botón "Ejecutar"
    submit.click()# clickea en "Ejecutar" y ejecuta consulta
    WebDriverWait(driver, 999999).until(lambda driver: len(driver.find_elements(By.TAG_NAME,'iframe')) == int(len(var1)))#espera 999 segundos o a que se muestren todas las tablas solicitadas, es decir, debe mostrar Descargar en formato Excel la misma cantidad de veces que la cantidad de variables de la solicitud
    html = driver.page_source
    suma = BeautifulSoup(html, "html.parser")
    i=0
    
    while i<=(len(var1)-1):   
        driver.switch_to.frame(i)
        WebDriverWait(driver, 999999).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(),'Descargar en formato Excel')]")))
        html = driver.page_source
        bs = BeautifulSoup(html, "html.parser")
        suma.body.append(bs)
        driver.switch_to.default_content()
        i+=1
    driver.close() # cierra navegador
    return suma # devuelve html con tabla de resultados

def make_dataframe(html): # crea dataframe con resultados
    temp_table = pd.read_html(str(html)) # lee todos los dataframes de la tabla de resultados
    return temp_table # devuelve lista con dataframes

merged = pd.concat(make_dataframe(make_query(query))) # concatena dataframes de la tabla de resultados en un solo dataframe
# Excelwriter = pd.ExcelWriter("tablas_redatam2007.xlsx",engine="xlsxwriter") #exporta en excel dataframe final
# merged.to_excel(Excelwriter)
# Excelwriter.save()

RUNDEF Job



TABLE TABLE1
    AS CROSSTABS OF Poblacio.PAREA BY Poblacio.P02SEXO
    AREABREAK Provinci
